En este codigo voy a buscar analizar los datos del test inicial. Basicamente ver si hay orientaciones mas faciles o mas dificiles que otras. 

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from scripts.db import *
from scripts.plots import *
from scripts.stats import *
from scripts.statsTransferencia import *
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

import pandas as pd

resumen = dataNumerica()
data = pd.DataFrame.from_dict(resumen, orient='index')

#Trasnformamos los valores que interesan en columnas (Por ahora voy a hacer todo en angulos)
dataTest = pd.DataFrame()
dataTestI = pd.DataFrame()
dataTestF = pd.DataFrame()
for i, col in enumerate(["P30", "P60", "P120","P150","A30","A60","A120","A150"]):
    dataTestI["I"+col] = data['TestInicialAngulos'].map(lambda x: x[i])
    dataTest["I"+col] = data['TestInicialAngulos'].map(lambda x: x[i])
for i, col in enumerate(["P30", "P60", "P120","P150","A30","A60","A120","A150"]):
    dataTestF["F"+col] = data['TestFinalAngulos'].map(lambda x: x[i])
    dataTest["F"+col] = data['TestFinalAngulos'].map(lambda x: x[i]) 

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:536: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:508: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:510: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)


In [31]:
pepe = dataTestI[['IP30', 'IP60','IP120', 'IP150']].stack().reset_index()
pepe.columns = ["Subject", "Exp", "Val"]
pepe['Extremo'] = 1
pepe.loc[(pepe['Exp'].str.contains("IP30")) | (pepe['Exp']=="IP150"), 'Extremo'] = "Extremo"
pepe.loc[(pepe['Exp'] == "IP60") | (pepe['Exp']=="IP120"), 'Extremo'] = "Medio"
pepe.head()

,Subject,Exp,Val,Extremo
0,Sujeto01,IP30,57.5,Extremo
1,Sujeto01,IP60,17.0,Medio
2,Sujeto01,IP120,34.0,Medio
3,Sujeto01,IP150,39.5,Extremo
4,Sujeto02,IP30,54.5,Extremo


In [33]:
results = smf.ols('Val ~ Exp', data=pepe).fit()

# Inspect the results
print(results.summary())

model = sm.MixedLM.from_formula("Val ~ Exp", pepe, groups=pepe["Subject"])

result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    Val   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     5.999
Date:                Fri, 04 Nov 2016   Prob (F-statistic):            0.00201
Time:                        20:12:53   Log-Likelihood:                -172.60
No. Observations:                  40   AIC:                             353.2
Df Residuals:                      36   BIC:                             360.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept       35.7500      6.034      5.924   

In [ ]:
from scipy.stats import ttest_rel

